<h1><center>Database CRUD</center></h1>

<center><img src="https://simplonline.co/_next/image?url=https%3A%2F%2Fsimplonline-v3-prod.s3.eu-west-3.amazonaws.com%2Fmedia%2Fimage%2Fjpg%2F8999e43e-b0c8-487c-93da-8edad1053392.jpg&w=1280&q=75" alt="data center" title="Data Center"  style="object-fit:cover; width:800px; height:250px;"/></center>

> __*Created by Charley Lebarbier*__

## Objectifs :  

Contexte du projet

Futurs développeur en intelligence artificielle, vous serez amenés la plupart du temps à récupérer, stocker puis analyser des données.  
Connaître le fonctionnemment des bases de données relationnelles est donc primordial...

Votre mission si vous l'acceptez :

__Dans phpMyAdmin__  
  
- I) Créer une base de données que vous appelerez : CountriesDatas  
- II) Vous ajouterez deux tables à de cette nouvelle base:  
  - a) countries (code, name)  
  - b) populations (id, LOCATION, TIME , Value)  

*NB: Pour l'id vous pourrez l'ajouter lorsque vous ferez les INSERTs,*  
*par exemple le 1er INSERT possède un id = 1, le 2ème id = 2, etc.*

​_______

__Dans un jupyter notebook__  
  
- III) Ensuite, vous devrez peupler les tables countries et populations 
respectivement à l'aide des fichiers suivants : country.txt et population.csv 

- IV) Enfin vous executerez une query de type SELECT sur la table population, 
en incluant un JOIN sur la table countries. Le but étant d'afficher les 
elements les "name" de la table countries au lieu des "LOCATION" de la table 
populations.  
  
*NB: Votre query devrait vous retourner une liste de tuple du type :*  
*[(217, 'Canada', datetime.date(1950, 1, 1), 14018600.0),*  
*(218, 'Canada', datetime.date(1951, 1, 1), 14318200.0),*  
*(219, 'Canada', datetime.date(1952, 1, 1), 14776300.0),*  
*(220, 'Canada', datetime.date(1953, 1, 1), 15169800.0),*  
*...]*  
  
- V) Pour finir, vous convertirez le resultat de la query précedente en dataframe

__________________________
__________________________

#### Import des librairies

In [79]:
# Import libraries
import pandas as pd
import mysql.connector as mysqlco
from mysql.connector import Error

#### Lecture des fichiers

In [80]:
# File reader : .txt or .csv
def read_file(file):
    """import a file (.txt or .csv) and prepare to inject in database"""

    try:
        # For .txt file
        if file.endswith('.txt'):
            # All data files are in folder 'data'
            with open(f'data/{file}', 'r') as f:
                file_content = []

                for row in f:
                    row_dict = {}

                    row = row.replace(' __****__ ', ' ')
                    row = row.replace('\n', '')

                    row_dict['code'] = row.strip('\n').split()[0]
                    row_dict['name'] = row.split(' ', 1)[1]

                    file_content.append(row_dict)
                # print(file_content)

        # For .csv file
        if file.endswith('.csv'):
            # All data files are in folder 'data'
            file_content = pd.read_csv(f'data/{file}')
            file_content.drop('Flag Codes', axis=1, inplace=True) 
            # print(file_content)
            
        return file_content


    except Exception as e:
        print(f"Error read_file : {e}")
        return False

countries = read_file('country.txt')
populations = read_file('population.csv')

In [81]:
# Check the variables containing the files

# print(countries)
# print('\n\n')
# print(populations)

#### Déclaration des info de connexion à la BDD

In [82]:
connection_config = {
                        "host" : "localhost",
                        "user" :"root",
                        "password" : "example",
                        "auth_plugin" : "mysql_native_password",
                        "database" : "CountriesDatas",
                        "port" : "3307"
                    }

#### Insertion des data 'country.txt' dans table 'Countries'

In [83]:
# Inject 'country' -> 'Countries' Table
def insert_data(config_co):
    """Inject data into a table"""
    try:
        db = mysqlco.connect(**config_co)
        cursor = db.cursor()
            
        for i, row in enumerate(countries):
            c_code = countries[i]['code']
            c_name = countries[i]['name']
        
            sql = f"""
                    INSERT INTO Countries (code, name)
                    VALUES ("{c_code}", "{c_name}")
                    ON DUPLICATE KEY UPDATE name = "{c_name}"; 
                   """
            cursor.execute(sql)
            db.commit()

        print("La table 'Countries' a été mise à jour.")
        return True


    except Exception as e:
        print(f"Error insert_data : {e}")
        return False
    
    
    finally:
        if db.is_connected():
            db.close
            cursor.close()

insert_data(connection_config)

La table 'Countries' a été mise à jour.


True

#### Insertion des data 'population.csv' -> 'Populations'

In [84]:
# Inject 'populations' -> 'Populations' Table
def insert_data(config_co):
    """Inject data into a table"""
    try:
        db = mysqlco.connect(**config_co)
        cursor = db.cursor()

        for index, data in populations.iterrows():
            # id, location, time, value
            sql = f"""
                    INSERT INTO Populations (id, location, time, value)
                    VALUES ({index}, "{data['LOCATION']}", 
                            {data['TIME']}, {data['Value']})
                    ON DUPLICATE KEY UPDATE location = "{data['LOCATION']}", 
                    time={data['TIME']}, value={data['Value']}; 
                   """
            cursor.execute(sql)
            db.commit()

        print("La table 'Populations' a été mise à jour.")
        return True


    except Exception as e:
        print(f"Error insert_data : {e}")
        return False
    
    
    finally:
        if db.is_connected():
            db.close
            cursor.close()

insert_data(connection_config)

La table 'Populations' a été mise à jour.


True

#### Retourner une selection sur table 'Populations' pour afficher une liste de tuples comme celle-ci : 

*(217, 'Canada', datetime.date(1950, 1, 1), 14018600.0),*  
*(218, 'Canada', datetime.date(1951, 1, 1), 14318200.0),*  
*(219, 'Canada', datetime.date(1952, 1, 1), 14776300.0),*  
*(220, 'Canada', datetime.date(1953, 1, 1), 15169800.0),*  
*...*  

In [91]:
def select_and_join(config_co):
    """Select some data in one table and join with another then display"""
    try:
        db = mysqlco.connect(**config_co)
        cursor = db.cursor()

        sql = f"""
                SELECT Populations.id, Countries.name, 
                       Populations.time, Populations.value
                FROM Populations
                INNER JOIN Countries ON Populations.location = Countries.code;
               """
        cursor.execute(sql)
        result = cursor.fetchall()

        return tuple(result)
        

    except Exception as e:
        print(f"Error select_and_join : {e}")
        return False


    finally:
        if db.is_connected():
            db.close()
            cursor.close()

pop_by_countries = select_and_join(connection_config)

In [ ]:
# Display the result

print(*pop_by_countries, sep='\n')

#### Conversion de notre resultat en dataframe

In [94]:
# Convert pop_by_countries in dataframe

df_Pop_Countries = pd.DataFrame(pop_by_countries, 
    columns = ['id', 'country_name', 'year', 'population'])

In [95]:
# Display the new dataframe

print(df_Pop_Countries)

          id country_name  year  population
0          0    Australia  1951    2.971132
1          1    Australia  1952    2.550554
2          2    Australia  1953    2.070283
3          3    Australia  1954    1.942078
4          4    Australia  1955    2.372448
...      ...          ...   ...         ...
12332  15627    Singapore  2016    0.790967
12333  15628    Singapore  2017    0.819538
12334  15629    Singapore  2018    0.718317
12335  15630    Singapore  2019    0.799292
12336  15631    Singapore  2020    0.447096

[12337 rows x 4 columns]


<center><h2>The End</h2></center>